In [1]:
import os
import sys
import random
from datetime import date, datetime

import pandas as pd
import numpy as np
import pyarrow as pa

import perspective

### Introduction to Perspective

Perspective is an interactive visualization component for large, real-time datasets. Originally developed for J.P. Morgan's trading business, Perspective makes it simple to build real-time & user configurable analytics in both the browser and in Python. Use it to create reports, dashboards, notebooks and applications, with static data or streaming updates via Apache Arrow.

In [2]:
# Set up some dummy data
data = pd.DataFrame({
    "a": np.random.rand(100),
    "b": np.ones(100),
    "c": [str(i) for i in range(100)],
    "d": [datetime.now() for i in range(100)],
    "e": [datetime.today() for i in range(100)]
})

# create a perspective.Table - the base container for data
table = perspective.Table(data)

# create a view - a query on the data
view = table.view(filter=[["a", ">", 0.5]], sort=[["a", "desc"]])

### Schemas

While Perspective can automatically infer data types, providing a schema offers explicit control over how your data appears and is treated.

For example, a column that has values such as `15.00` might be inferred as a `float`, but treating it as an `int` would look cleaner on the grid.

In [3]:
# Create an indexed table from a schema - a mapping of column names to types, primary keyed by `a`
schema_table = perspective.Table({
    "a": float,
    "b": float,
    "c": str,
    "d": datetime
}, index="a")

### Updates

To append data or make in-place updates, call the Table's `update()` method with the dataset you want to add.

If the Table has an `index` set, like `schema_table` above, updates will occur in-place based on the index column. If the Table is unindexed, updates will append
data to the end of the Table.

In [4]:
# Update the unindexed table
table.update([
    {"a": 1500, "b": 1, "c": "string", "d": datetime.now()},
    {"a": 2500, "b": 2, "c": "string2", "d": datetime.now()}
])

# Query it to see the new data
sorted_view = table.view(sort=[["a", "asc"]])

sorted_view.to_df()

,index,a,b,c,d,e
0,64,0.003827,1.0,64,2020-09-10 11:17:21.664,2020-09-10 11:17:21.664
1,8,0.019815,1.0,8,2020-09-10 11:17:21.664,2020-09-10 11:17:21.664
2,47,0.057882,1.0,47,2020-09-10 11:17:21.664,2020-09-10 11:17:21.664
3,49,0.061360,1.0,49,2020-09-10 11:17:21.664,2020-09-10 11:17:21.664
4,91,0.086733,1.0,91,2020-09-10 11:17:21.664,2020-09-10 11:17:21.664
...,...,...,...,...,...,...
97,98,0.965290,1.0,98,2020-09-10 11:17:21.664,2020-09-10 11:17:21.664
98,45,0.981582,1.0,45,2020-09-10 11:17:21.664,2020-09-10 11:17:21.664
99,72,0.998413,1.0,72,2020-09-10 11:17:21.664,2020-09-10 11:17:21.664
100,None,1500.000000,1.0,string,2020-09-10 15:17:22.610,NaT


In [5]:
# On indexed tables, updates with new primary keys will append
schema_table.update([
    {"a": 1.5, "b": 100, "c": "a"},
    {"a": 2.5, "b": 150, "c": "b"},
    {"a": 3.5, "b": 200, "c": "c"},
])

schema_view = schema_table.view()
schema_view.to_df()

,a,b,c,d
0,1.5,100.0,a,None
1,2.5,150.0,b,None
2,3.5,200.0,c,None


In [6]:
# Updates with the same primary keys will overwrite
schema_table.update([
    {"a": 5.5, "b": 150, "c": "appended!"},
    {"a": 1.5, "b": 1500, "c": "overwritten!"},
])

schema_view = schema_table.view()
schema_view.to_df()

,a,b,c,d
0,1.5,1500.0,overwritten!,None
1,2.5,150.0,b,None
2,3.5,200.0,c,None
3,5.5,150.0,appended!,None


### `on_update` callbacks

Using `on_update`, you can execute a callback whenever the Table updates. This offers a world of possibilities throgh chaining together Perspective Tables, triggering actions on a UI, making calculations whenever the Table has updated, etc.

Here, we use `on_update` to update an indexed table from an unindexed table:

In [7]:
# Add an `on_update` callback, which fires with an Arrow-encoded binary of the updated rows
def callback(port, delta):
    # Update the indexed table with the updated rows, which will update in-place based on `index`
    schema_table.update(delta)

view.on_update(callback, mode="row")

In [8]:
# Update the unindexed table, and query the indexed table
# On indexed tables, updates with new primary keys will append
table.update([
    {"a": 1.5, "b": 100, "c": "just updated"},
    {"a": 2.5, "b": 150, "c": "also new"},
    {"a": 3.5, "b": 200, "c": "me three"},
    {"a": 6.5, "b": 200, "c": "just appended"},
])

schema_view.to_df()

,a,b,c,d
0,1.5,100.0,just updated,None
1,2.5,150.0,also new,None
2,3.5,200.0,me three,None
3,5.5,150.0,appended!,None
4,6.5,200.0,just appended,None


### Serializing Data

Data can be retrieved from a `View` by calling one of its `to_*` methods: `to_df`, `to_arrow`, `to_dicts`, `to_records`, and `to_csv`. When the underlying `Table` updates, the `View` is automatically notified with new data and will always return the most up-to-date dataset.

In [9]:
# get some data from the view - supported formats are dataframes, Apache Arrow, dicts of numpy arrays, row/columnar JSON, and CSV
filtered = view.to_df(end_row=10)
display(filtered)

,index,a,b,c,d,e
0,None,2500.000000,2.0,string2,2020-09-10 15:17:22.610,NaT
1,None,1500.000000,1.0,string,2020-09-10 15:17:22.610,NaT
2,None,6.500000,200.0,just appended,NaT,NaT
3,None,3.500000,200.0,me three,NaT,NaT
4,None,2.500000,150.0,also new,NaT,NaT
5,None,1.500000,100.0,just updated,NaT,NaT
6,72,0.998413,1.0,72,2020-09-10 11:17:21.664,2020-09-10 11:17:21.664
7,45,0.981582,1.0,45,2020-09-10 11:17:21.664,2020-09-10 11:17:21.664
8,98,0.965290,1.0,98,2020-09-10 11:17:21.664,2020-09-10 11:17:21.664
9,69,0.958856,1.0,69,2020-09-10 11:17:21.664,2020-09-10 11:17:21.664


In [10]:
# append to the table with new data
table.update([{"a": 1.5, "b": 1, "c": "string", "d": datetime.now()}])

# re-query the data
filtered = view.to_df(end_row=10)
display(filtered)

,index,a,b,c,d,e
0,None,2500.000000,2.0,string2,2020-09-10 15:17:22.610,NaT
1,None,1500.000000,1.0,string,2020-09-10 15:17:22.610,NaT
2,None,6.500000,200.0,just appended,NaT,NaT
3,None,3.500000,200.0,me three,NaT,NaT
4,None,2.500000,150.0,also new,NaT,NaT
5,None,1.500000,100.0,just updated,NaT,NaT
6,None,1.500000,1.0,string,2020-09-10 15:17:25.840,NaT
7,72,0.998413,1.0,72,2020-09-10 11:17:21.664,2020-09-10 11:17:21.664
8,45,0.981582,1.0,45,2020-09-10 11:17:21.664,2020-09-10 11:17:21.664
9,98,0.965290,1.0,98,2020-09-10 11:17:21.664,2020-09-10 11:17:21.664


### Working with Arrows  

Perspective can load and create binaries in the Apache Arrow format - just call the `View`'s `to_arrow()` method. After the arrow is emitted, it can be saved to disk, coerced into a `pyarrow.Table`, and more. 

In [11]:
arrow = view.to_arrow()
with open("example.arrow", "wb") as file:
    file.write(arrow)

Opening arrows from the filesystem is easy:

In [12]:
with open("example.arrow", "rb") as arrow:
    w = perspective.PerspectiveWidget(arrow.read(), column_pivots=["c"])
    display(w)

PerspectiveWidget(column_pivots=['c'], columns=['index', 'a', 'b', 'c', 'd', 'e'])

### PerspectiveWidget

`PerspectiveWidget` is a Jupyter widget that uses `<perspective-viewer>` to visualize and transform your dataset within the browser, with all the same features and performance as Perspective in a standalone web application.

In [23]:
# Create a PerspectiveWidget from the tables we just created
widget = perspective.PerspectiveWidget(schema_table, plugin="datagrid", row_pivots=["a"], aggregates={"a": "avg"}, editable=True)
display(widget)

PerspectiveWidget(aggregates={'a': 'avg'}, columns=['a', 'b', 'c', 'd'], editable=True, row_pivots=['a'])

### Streaming data in Perspective

We've demonstrated how Perspective can be used with static datasets and schemas, but let's create a streaming example that shows how it performs over rapidly streaming data. The following datasource generates random rows every 50 milliseconds, and runs on an IOloop:

In [24]:
import asyncio
import threading

In [25]:
SECURITIES = ["AAPL.N", "AMZN.N", "QQQ.N", "NVDA.N", "TSLA.N",
              "FB.N", "MSFT.N", "TLT.N", "XIV.N", "YY.N",
              "CSCO.N", "GOOGL.N", "PCLN.N"]

CLIENTS = ["Homer", "Marge", "Bart", "Lisa", "Maggie",
           "Moe", "Lenny", "Carl", "Krusty"]

def data_source():
    rows = []
    modifier = random.random() * random.randint(1, 50)
    for i in range(5):
        rows.append({
            "name": SECURITIES[random.randint(0, len(SECURITIES) - 1)],
            "client": CLIENTS[random.randint(0, len(CLIENTS) - 1)],
            "open": (random.random() * 75 + random.randint(0, 9)) * modifier,
            "high": (random.random() * 105 + random.randint(1, 3)) * modifier,
            "low": (random.random() * 85 + random.randint(1, 3)) * modifier,
            "close": (random.random() * 90 + random.randint(1, 3)) * modifier,
            "lastUpdate": datetime.now(),
            "date": date.today()
        })
    return rows

In [26]:
streaming_table = perspective.Table({
    "name": str,
    "client": str,
    "open": float,
    "high": float,
    "low": float,
    "close": float,
    "lastUpdate": datetime,
    "date": date
}, limit=2500)

In [31]:
streaming_widget = perspective.PerspectiveWidget(
    streaming_table,
    plugin="d3_candlestick",
    row_pivots=["lastUpdate"],
    columns=["open", "close", "high", "low"]
)

In [32]:
streaming_widget

PerspectiveWidget(columns=['open', 'close', 'high', 'low'], plugin='d3_candlestick', row_pivots=['lastUpdate']…

In [29]:
async def _update():
    while True:
        streaming_table.update(data_source())
        await asyncio.sleep(0.05)
        
def update_table():
    loop = asyncio.new_event_loop()
    task = loop.create_task(_update())
    loop.call_later(60, task.cancel)
    
    try:
        loop.run_until_complete(task)
    except asyncio.CancelledError:
        print("Stopped streaming!")
        pass

In [30]:
thread = threading.Thread(target=update_table)
thread.start()

In [33]:
thread.join()